<a href="https://colab.research.google.com/github/WhiteAndBlackFox/LearningPyTorch/blob/CNN/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN свертки

Здача будет разбита на 3 пункта:
* Обучить CNN (самописная) на CIFAR-100.
* Обучить CNN на CIFAR-100 через дообучение ImageNet Resnet-50.
* Обучить CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.



## Подключаем библиотеки

In [1]:
import os
import pickle
from PIL import Image

import numpy as np

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms, datasets, models
from torchvision.datasets.utils import download_and_extract_archive, check_integrity

### Глобальные переменные

In [12]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

STATS = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

EPOCHES = 5

PATH_DOWNLOAD = 'data/'

In [13]:
print(DEVICE)

cpu


## Реализация

### Обучить CNN (самописная) на CIFAR-100

#### Создадим Dataset

In [3]:
class Cifar100Dataset(Dataset):
   
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]

    meta = {
        'filename': 'meta',
        'key': 'fine_label_names',
        'md5': '7973b15100ade9c7d40fb424638fde48',
    }

    err_msg = "Dataset поврежден или не найден. Надо указать флаг download=True"

    def __init__(self, root, train=True, transform=None, download=False):

        self.data = []
        self.targets = []

        self.train = train
        self.root = root
        self.transform = transform
        
        if download:
          self.download()

        if not self._check_data_cifar():
          raise RuntimeError(self.err_msg)

        if self.train:
          download_list = self.train_list
        else:
          download_list = self.test_list

        for file_name, md5_file in download_list:
          file_path = os.path.join(self.root, self.base_folder, file_name)
          with open(file_path, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
            self.data.append(pickle_data['data'])
            if 'labels' in pickle_data:
              self.targets.extend(pickle_data['labels'])
            else:
              self.targets.extend(pickle_data['fine_labels'])

        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))
        

    def _load_meta(self):
      path = os.path.join(self.root, self.base_folder, self.meta['filename'])
      if not check_integrity(path, self.meta['md5']):
        raise RuntimeError(self.err_msg)
      with open(path, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        self.classes = data[self.meta['key']]
      self.class_to_idx = { _class: i for i, _class in enumerate(self.classes) }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
       img, target = self.data[idx], self.targets[idx]
       img = Image.fromarray(img)
       if self.transform:
         img = self.transform(img)
       return img, target
    
    def _check_data_cifar(self):
      for fentry in (self.train_list + self.test_list):
          filename, md5 = fentry[0], fentry[1]
          fpath = os.path.join(self.root, self.base_folder, filename)
          if not check_integrity(fpath, md5):
              return False
      return True

    def download(self):
      if self._check_data_cifar():
        print("Файлы актуальны")
        return
      download_and_extract_archive(self.url, self.root, filename=self.filename, md5=self.tgz_md5)

#### Опишем нейронную сеть

In [4]:
class Cifar100Net(nn.Module):
  def __init__(self) -> None:
      super(Cifar100Net, self).__init__()
      self.block_1 = nn.Sequential(
           nn.Conv2d(in_channels=3, out_channels=30, kernel_size=(3), stride=(1), padding=1, bias=False),
           nn.ReLU(),
           nn.BatchNorm2d(30),
           nn.Dropout2d(0.2),
           nn.MaxPool2d(kernel_size=2, stride=2))
      
      self.block_2 = nn.Sequential(
           nn.Conv2d(in_channels=30, out_channels=60, kernel_size=(3), stride=(1), padding=1, bias=False),
           nn.ReLU(),
           nn.BatchNorm2d(60),
           nn.Dropout2d(0.2),
           nn.MaxPool2d(kernel_size=2, stride=2))

      self.block_3 = nn.Sequential(
           nn.Conv2d(in_channels=60, out_channels=120, kernel_size=(3), stride=(1), padding=1, bias=False),
           nn.ReLU(),
           nn.BatchNorm2d(120),
           nn.Dropout2d(0.2),
           nn.MaxPool2d(kernel_size=2, stride=2))
      
      # Планировалось в 4 блока сделать, но памяти не хватило
      # self.block_4 = nn.Sequential(
      #      nn.Conv2d(in_channels=120, out_channels=240, kernel_size=(3), stride=(1), padding=1, bias=False),
      #      nn.ReLU(),
      #      nn.BatchNorm2d(240),
      #      nn.Dropout2d(0.2),
      #      nn.MaxPool2d(kernel_size=2, stride=2))
      
      self.predict = nn.Sequential(
          # Из-за памяти и пришлось сделать еще один слой переходящий из 1920 в 960
          nn.Linear(1920, 960),
          nn.Linear(960, 400),
          nn.Linear(400, 100)
      )

  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    # out = self.block_4(out)
    out = out.view(out.size(0), -1)
    out = self.predict(out)
    return out

#### Функция для проверки

In [5]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.CrossEntropyLoss()
  best_acc = {'train': None, 'test': None}
  net.train()
  for epoch in range(EPOCHES):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)

        # обнуляем градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 200 == 0 or (i + 1) == len(train_loader):
            net.eval()

            test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
            for y, (out_test, lbl_test) in enumerate(test_loader):
                test_outputs = net(out_test)
                test_loss += loss_fn(test_outputs, lbl_test)
                test_running_total += len(lbl_test)
            
            res_loss_train = running_loss / running_items
            res_loss_test = test_loss / test_running_total
            
            if best_acc['train'] is None or res_loss_train < best_acc['train']:
              best_acc['train'] = res_loss_train
            
            if best_acc['test'] is None or res_loss_test < best_acc['test']:
              best_acc['test'] = res_loss_train

            print(f'Epoch [{epoch + 1}/{EPOCHES}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {res_loss_train:.3f}. '\
                  f'Test acc: {res_loss_test:.3f}.')
            
            running_loss, running_items = 0.0, 0.0
            net.train()
  print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
  print('Training is finished!')

#### Начнем для самописной анализ

In [6]:
# Трансформация для тренеровочной выборки и для тестовой выборки
train_transforms = transforms.Compose([transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         transforms.RandomHorizontalFlip(), 
                         transforms.ToTensor(), 
                         transforms.Normalize(*STATS, inplace=True)])

valid_transforms = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize(*STATS)])

In [7]:
# Собираем данные для обучения train=True
train_dataset = Cifar100Dataset(root=PATH_DOWNLOAD,
                                  train=True,
                                  transform=train_transforms,
                                  download=True)
# Загружаем данные, для дальнейшего их обучения в тренеровочную сетку
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

4.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

7.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11.1%IOPub me

In [8]:
# проделываем то же самое для тестовых данных train=False
test_dataset = Cifar100Dataset(root=PATH_DOWNLOAD, train=False,
                                       download=True, transform=valid_transforms)
test_loader = DataLoader(test_dataset, batch_size=64,
                                         shuffle=False)

Файлы актуальны


In [9]:
net = Cifar100Net().to(DEVICE)
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [10]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/5]. Step [1/782]. Loss: 0.072. Test acc: 0.231.
Epoch [1/5]. Step [257/782]. Loss: 0.314. Test acc: 0.070.
Epoch [1/5]. Step [513/782]. Loss: 0.070. Test acc: 0.065.
Epoch [1/5]. Step [769/782]. Loss: 0.067. Test acc: 0.061.
Epoch [1/5]. Step [782/782]. Loss: 0.070. Test acc: 0.061.
Epoch [2/5]. Step [1/782]. Loss: 0.066. Test acc: 0.062.
Epoch [2/5]. Step [257/782]. Loss: 0.065. Test acc: 0.060.
Epoch [2/5]. Step [513/782]. Loss: 0.061. Test acc: 0.055.
Epoch [2/5]. Step [769/782]. Loss: 0.058. Test acc: 0.053.
Epoch [2/5]. Step [782/782]. Loss: 0.060. Test acc: 0.053.
Epoch [3/5]. Step [1/782]. Loss: 0.057. Test acc: 0.053.
Epoch [3/5]. Step [257/782]. Loss: 0.056. Test acc: 0.052.
Epoch [3/5]. Step [513/782]. Loss: 0.055. Test acc: 0.051.
Epoch [3/5]. Step [769/782]. Loss: 0.055. Test acc: 0.050.
Epoch [3/5]. Step [782/782]. Loss: 0.060. Test acc: 0.050.
Epoch [4/5]. Step [1/782]. Loss: 0.054. Test acc: 0.050.
Epoch [4/5]. Step [257/782]. Loss: 0.055. Test acc: 0.051.
Epoch

### Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50  с аугментацией данных.

In [22]:
net = models.resnet50(pretrained=True)

In [23]:
for param in net.parameters():
    param.requires_grad = False

In [24]:
net.fc = nn.Linear(2048, 100)

In [25]:
# Трансформация для тренеровочной выборки и для тестовой выборки
train_transforms = transforms.Compose([transforms.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         transforms.RandomHorizontalFlip(), 
                         transforms.ToTensor(), 
                         transforms.Normalize(*STATS, inplace=True)])

valid_transforms = transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize(*STATS)])

In [26]:
# Собираем данные для обучения train=True
train_dataset = Cifar100Dataset(root=PATH_DOWNLOAD,
                                  train=True,
                                  transform=train_transforms,
                                  download=True)
# Загружаем данные, для дальнейшего их обучения в тренеровочную сетку
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

Файлы актуальны


In [28]:
params_to_update = []
for name,param in net.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=0.001)

In [29]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/5]. Step [1/782]. Loss: 0.074. Test acc: 0.079.
Epoch [1/5]. Step [257/782]. Loss: 0.061. Test acc: 0.055.
Epoch [1/5]. Step [513/782]. Loss: 0.055. Test acc: 0.054.
Epoch [1/5]. Step [769/782]. Loss: 0.053. Test acc: 0.052.
Epoch [1/5]. Step [782/782]. Loss: 0.057. Test acc: 0.053.
Epoch [2/5]. Step [1/782]. Loss: 0.051. Test acc: 0.053.
Epoch [2/5]. Step [257/782]. Loss: 0.051. Test acc: 0.051.
Epoch [2/5]. Step [513/782]. Loss: 0.052. Test acc: 0.051.
Epoch [2/5]. Step [769/782]. Loss: 0.051. Test acc: 0.050.
Epoch [2/5]. Step [782/782]. Loss: 0.057. Test acc: 0.051.
Epoch [3/5]. Step [1/782]. Loss: 0.056. Test acc: 0.050.
Epoch [3/5]. Step [257/782]. Loss: 0.050. Test acc: 0.052.
Epoch [3/5]. Step [513/782]. Loss: 0.050. Test acc: 0.050.
Epoch [3/5]. Step [769/782]. Loss: 0.050. Test acc: 0.051.
Epoch [3/5]. Step [782/782]. Loss: 0.054. Test acc: 0.049.
Epoch [4/5]. Step [1/782]. Loss: 0.041. Test acc: 0.049.
Epoch [4/5]. Step [257/782]. Loss: 0.049. Test acc: 0.050.
Epoch